-----
# cartoGRAPHs - FIGURE 6B
-----

In [ ]:
from cartoGRAPHs import * 

_____ 
# HUMAN NETWORK
_____

In [2]:
organism = 'human'

G = load_graph(organism) 

d_gene_sym = load_genesymbols(G, organism)
l_features = list(d_gene_sym.values())

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame(d_centralities).T
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
essential_genes,non_ess_genes,notdefined_genes = load_essentiality(G, organism)

# get Layout basis --> global layout matrix 
DF_structural = load_datamatrix(G,organism, 'global')
df_max = DF_structural.max()
l_max_visprob = max(list(df_max.values))

# get Disease Feature Matrix
FM_disease = pd.read_pickle('input/Features_Disease_Dataframe_'+organism+'.pickle')

---
# VECTOR : NF1 GENE + DISEASE MODULE (8712 - neurofibromatosis)
---
e.g.
+ DOID:8712 / Neurofibromatosis
+ DOID:962 / Neurofibroma
+ DOID:0050736 / Autosomal Dominant Disease

In [46]:
path_nf = 'input/neurofibromatosis/'

# gene of interest
nf_goi_ = str(int(np.loadtxt(path_nf+'nf_gene_of_interest.txt', unpack=False)))
nf_goi = list(nf_goi_.split(" "))
nf_goi_str = ' '.join([str(item) for item in nf_goi])

nf_neighbors = []
for edge in G.edges():
    if edge[0] == nf_goi_str:
        nf_neighbors.append(edge[1])
    elif edge[1] == nf_goi_str: 
        nf_neighbors.append(edge[0])

nf_all = nf_neighbors + nf_goi

# get disease module genes 
num_doid = '8712' 

# define specific DOID 
dismod = FM_disease.loc[:,'DOID:'+num_doid]
dismod.index = FM_disease.index
dismod_id = 'DOID'+num_doid

dismod_genes = [] 
for i,v in enumerate(dismod.values):
    if v == 1:
        dismod_genes.append(dismod.index[i])
    else:
        pass

l_genes_nf = [str(i) for i in dismod_genes]
print('Number of Genes associated to DOID', num_doid,':',len(l_genes_nf))
l_genes_neighbors = [str(i) for i in nf_neighbors]
print('Number of NF1 neighbor genes', num_doid,':',len(l_genes_neighbors))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod = pd.DataFrame(dismod.reindex(DF_structural.index.copy() ,fill_value=0))
DF_dismod_withneigh = pd.DataFrame(dismod.reindex(DF_structural.index.copy() ,fill_value=0))

for i in DF_dismod.index:
    if i in l_genes_nf:
        DF_dismod.loc[i] = 1
        DF_dismod_withneigh.loc[i] = 1
    elif i in l_genes_neighbors:
        DF_dismod_withneigh.loc[i] = 1
    else: 
        pass

DF_dismod.columns = ['Neurofib 8712']
DF_dismod_withneigh.columns = ['Neurofib 8712+neigh']

DF_NFneighbors = pd.DataFrame(dismod.reindex(DF_structural.index.copy() ,fill_value=0))
for i in DF_NFneighbors.index:
    if i in l_genes_neighbors:# and i != nf_goi_str:
        DF_NFneighbors.loc[i] = 1
    
DF_NFneighbors.columns = ['NF1 neigh']

Number of Genes associated to DOID 8712 : 12
Number of NF1 neighbor genes 8712 : 49


---
# VECTORS : Disease Context 
---
+ DOID:3165 - Skin benign neoplasm
+ DOID:10534 - Stomach Cancer
+ DOID:3620 - Central Nervous System Cancer
+ DOID:0060089 - Endocrine organ benign neoplasm
+ resources: https://disease-ontology.org/ https://en.wikipedia.org/wiki/Neurofibromatosis

### First Disease Vector | DOID:3165 - Skin benign neoplasm

In [47]:
# -----------------------------
num_doid_1 = '3165'
# -----------------------------

# define specific DOID 
dismod_1 = FM_disease.loc[:,'DOID:'+num_doid_1]
dismod_1.index = FM_disease.index

# get all genes associated to disease
dismod_genes_1 = [] 
for i,v in enumerate(dismod_1.values):
    if v == 1:
        dismod_genes_1.append(dismod_1.index[i])
    else:
        pass

l_genes_dismod_1 = [str(i) for i in dismod_genes_1]
l_genes_nfdismod_overlap_1 = []
for i in nf_neighbors:
    if i in l_genes_dismod_1:
        l_genes_nfdismod_overlap_1.append(i)
l_genes_dismod_1_all = l_genes_dismod_1 + l_genes_nfdismod_overlap_1
print('Number of Genes associated to DOID', num_doid_1,':',len(l_genes_dismod_1))
print('Number of Genes associated to DOID with overlap of NF neighbors', num_doid_1,':',len(l_genes_dismod_1_all))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_1 = pd.DataFrame(dismod_1.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_1.index:
    if i in l_genes_dismod_1_all: #l_genes_dismod_1:
        DF_dismod_1.loc[i] = 1

Number of Genes associated to DOID 3165 : 163
Number of Genes associated to DOID with overlap of NF neighbors 3165 : 165


### Second Disease Vector | DOID:10534 - Stomach Cancer

In [48]:
# -----------------------------
num_doid_2 = '10534' #'3883' #'1612' #'3490'
# -----------------------------

# define specific DOID 
dismod_2 = FM_disease.loc[:,'DOID:'+num_doid_2]
dismod_2.index = FM_disease.index

# get all genes associated to disease
dismod_genes_2 = [] 
for i,v in enumerate(dismod_2.values):
    if v == 1:
        dismod_genes_2.append(dismod_2.index[i])
    else:
        pass
    
l_genes_dismod_2 = [str(i) for i in dismod_genes_2]
l_genes_nfdismod_overlap_2 = []
for i in nf_neighbors:
    if i in l_genes_dismod_2:
        l_genes_nfdismod_overlap_2.append(i)
l_genes_dismod_2_all = l_genes_dismod_2 + l_genes_nfdismod_overlap_2
print('Number of Genes associated to DOID', num_doid_2,':',len(l_genes_dismod_2)) 
print('Number of Genes associated to DOID with overlap of NF neighbors', num_doid_2,':',len(l_genes_dismod_2_all))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_2 = pd.DataFrame(dismod_2.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_2.index:
    if i in l_genes_dismod_2_all:
        DF_dismod_2.loc[i] = 1

Number of Genes associated to DOID 10534 : 303
Number of Genes associated to DOID with overlap of NF neighbors 10534 : 307


### Third Disease Vector | DOID:3620 - Central Nervous System Cancer

In [49]:
# -----------------------------
num_doid_3 = '3620' #'0060115' 
# -----------------------------

# define specific DOID 
dismod_3 = FM_disease.loc[:,'DOID:'+num_doid_3]
dismod_3.index = FM_disease.index

dismod_genes_3 = [] 
for i,v in enumerate(dismod_3.values):
    if v == 1:
        dismod_genes_3.append(dismod_3.index[i])
    else:
        pass

l_genes_dismod_3 = [str(i) for i in dismod_genes_3]
l_genes_nfdismod_overlap_3 = []
for i in nf_neighbors:
    if i in l_genes_dismod_3:
        l_genes_nfdismod_overlap_3.append(i)
l_genes_dismod_3_all = l_genes_dismod_3 + l_genes_nfdismod_overlap_3
print('Number of Genes associated to DOID', num_doid_3,':',len(l_genes_dismod_3))
print('Number of Genes associated to DOID with overlap of NF neighbors', num_doid_3,':',len(l_genes_dismod_3_all))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_3 = pd.DataFrame(dismod_3.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_3.index:
    if i in l_genes_dismod_3_all:
        DF_dismod_3.loc[i] = 1

Number of Genes associated to DOID 3620 : 173
Number of Genes associated to DOID with overlap of NF neighbors 3620 : 174


### Fourth Disease Vector | DOID:0060089 - Endocrine Organ benign neoplasm

In [50]:
# -----------------------------
num_doid_4 = '0060089'
# -----------------------------

# define specific DOID 
dismod_4 = FM_disease.loc[:,'DOID:'+num_doid_4]
dismod_4.index = FM_disease.index
dismod_id_4 = 'DOID'+num_doid_4

dismod_genes_4 = [] 
for i,v in enumerate(dismod_4.values):
    if v == 1:
        dismod_genes_4.append(dismod_4.index[i])
    else:
        pass
    
l_genes_dismod_4 = [str(i) for i in dismod_genes_4]
l_genes_nfdismod_overlap_4 = []
for i in nf_neighbors:
    if i in l_genes_dismod_4:
        l_genes_nfdismod_overlap_4.append(i)
l_genes_dismod_4_all = l_genes_dismod_4 + l_genes_nfdismod_overlap_4
print('Number of Genes associated to DOID', num_doid_4,':',len(l_genes_dismod_4))
print('Number of Genes associated to DOID with overlap of NF neighbors', num_doid_4,':',len(l_genes_dismod_4_all))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_4 = pd.DataFrame(dismod_4.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_4.index:
    if i in l_genes_dismod_4_all:
        DF_dismod_4.loc[i] = 1

Number of Genes associated to DOID 0060089 : 122
Number of Genes associated to DOID with overlap of NF neighbors 0060089 : 123


---
# ALL DISEASE VECTORS / MATRIX PREP 
---

In [62]:
# --------------------------------------
#
scalar_val = 2 
#
# -------------------------------------- 
enhance_factor_discontext = round((1-l_max_visprob)*scalar_val,10)
print('ENHANCING FACTOR Disease Context : ', enhance_factor_discontext)

# Vector of NF1 Neurofibromatosis
DF_dismod_scal = DF_dismod * enhance_factor_discontext
DF_dismod_withneigh_scal = DF_dismod_withneigh * enhance_factor_discontext

# Vector of NF1 neighbor genes 
DF_NFneighbors_scal = DF_NFneighbors * enhance_factor_discontext 

DF_dismod_1_scal = DF_dismod_1 * enhance_factor_discontext
DF_dismod_2_scal = DF_dismod_2 * enhance_factor_discontext
DF_dismod_3_scal = DF_dismod_3 * enhance_factor_discontext
DF_dismod_4_scal = DF_dismod_4 * enhance_factor_discontext

# merge into one dataframe 
DF_merge_discontext = pd.concat([
                                DF_structural, # original global matrix 
                                
                                DF_dismod_scal,  # Disease Module Neurofibromatosis
                                #DF_dismod_withneigh_scal, # Disease Module Neurofibromatosis WITH NEIGHBORS of NF1 gene
                                #DF_NFneighbors_scal, #_scal, # Neighbors of NF1
                        
                                DF_dismod_1_scal, # Disease Context Vectors 1-4
                                DF_dismod_2_scal,
                                DF_dismod_3_scal,
                                DF_dismod_4_scal,
    
                                ],
                                axis=1
                                )

#DF_merge_discontext

ENHANCING FACTOR Disease Context :  0.187514


_____ 
# SPATIAL EMBEDDING
_____

### 2D PORTRAIT POSITIONING 

In [63]:
n_n = 8 #6 #20
spr = 1.0
md = 0.0  #.4
metr = 'cosine'

In [64]:
%%time 
posG_umap2D = layout_portrait_umap(G,DF_merge_discontext, 2, 
                                   n_neighbors = n_n, 
                                   spread = spr, 
                                   min_dist = md, 
                                   metric = metr) 
posG = posG_umap2D

CPU times: user 1min 19s, sys: 7.81 s, total: 1min 27s
Wall time: 43.2 s


In [65]:
#%%time 
#posG_umap3D = layout_portrait_umap(G,DF_merge_discontext, 3, 
#                                   n_neighbors = n_n, 
#                                   spread = spr, 
#                                   min_dist = md, 
#                                   metric = metr) 
#posG = posG_umap3D

____________
# VISUAL SETTINGS + PLOT PREPARATION
____________

### DISEASE SPECIFIC

In [66]:
#
# DISEASE 1 | ORANGE
#

# color nodes
node_col_1 = '#FF7001' # (orange) #'#B00160' 

center_dis_1_sym = 'TYR' #'GLI2' #'AIRE' # 'KRAS' #'PTEN' 

# get nodeID of gene symbol 
for ix, sym in d_gene_sym.items():
    if sym == center_dis_1_sym:
        center_dis_1_id = ix

# get coordinates from nodeID
for node, coords in posG.items():
    if node == center_dis_1_id:
        center_dis_1_coords = coords

# make shapely point from defined center of cluster 
point_1 = geometry.Point(center_dis_1_coords[0], center_dis_1_coords[1])

# create your circle buffer from one of the points
distance = 0.05

circle_buffer = point_1.buffer(distance)

# get list of genes that are within radius around the center 
l_genes_circlebuffer_1 = []
for node,coords in posG.items():
    point = geometry.Point(coords[0], coords[1])
    if point.within(circle_buffer) and node in l_genes_dismod_1: 
        l_genes_circlebuffer_1.append(node)
        
print('Number of genes within radius around Center - Disease 1:' ,len(l_genes_circlebuffer_1))

# color edges within circle buffer     
d_edge_col_circlebuffer_1 = color_edges_from_nodelist_specific(G, l_genes_circlebuffer_1, node_col_1)
print('Number of links in circle buffer - Disease 1:', len(d_edge_col_circlebuffer_1))

# color all dismod edges    
d_edge_col_1 = color_edges_from_nodelist_specific(G, l_genes_dismod_1, node_col_1)

Number of genes within radius around Center - Disease 1: 91
Number of links in circle buffer - Disease 1: 105


In [67]:
#
# DISEASE 2 | YELLOW
#

# color nodes
node_col_2 = '#FFC433' #(yellow)

center_dis_2_sym = 'ACTC1' #'HIKESHI' # 'ZNF160' #'TBX3' 
# get nodeID of gene symbol 
for ix, sym in d_gene_sym.items():
    if sym == center_dis_2_sym:
        center_dis_2_id = ix

# get coordinates from nodeID
for node, coords in posG.items():
    if node == center_dis_2_id:
        center_dis_2_coords = coords

# make shapely point from defined center of cluster 
point_2 = geometry.Point(center_dis_2_coords[0], center_dis_2_coords[1])

# create your circle buffer from one of the points
distance = 0.05
circle_buffer = point_2.buffer(distance)

# get list of genes that are within radius around the center 
l_genes_circlebuffer_2 = []
for node,coords in posG.items():
    point = geometry.Point(coords[0], coords[1])
    if point.within(circle_buffer) and node in l_genes_dismod_2: 
        l_genes_circlebuffer_2.append(node)
        
print('Number of genes within radius around Center - Disease 2:' ,len(l_genes_circlebuffer_2))

# color edges within circle buffer     
d_edge_col_circlebuffer_2 = color_edges_from_nodelist_specific(G, l_genes_circlebuffer_2, node_col_2)
print('Number of links in circle buffer - Disease 2:', len(d_edge_col_circlebuffer_2))

# color all dismod edges    
d_edge_col_2 = color_edges_from_nodelist_specific(G, l_genes_dismod_2, node_col_2)

Number of genes within radius around Center - Disease 2: 194
Number of links in circle buffer - Disease 2: 165


In [68]:
#
# DISEASE 3 | GREEN
#

# color nodes
node_col_3 = '#BEDC00' # (yellowgreen)

center_dis_3_sym = 'FLI1' #'MTR' 
# get nodeID of gene symbol 
for ix, sym in d_gene_sym.items():
    if sym == center_dis_3_sym:
        center_dis_3_id = ix

# get coordinates from nodeID
for node, coords in posG.items():
    if node == center_dis_3_id:
        center_dis_3_coords = coords

# make shapely point from defined center of cluster 
point_3 = geometry.Point(center_dis_3_coords[0], center_dis_3_coords[1])

# create your circle buffer from one of the points
distance = 0.06
circle_buffer = point_3.buffer(distance)

# get list of genes that are within radius around the center 
l_genes_circlebuffer_3 = []
for node,coords in posG.items():
    point = geometry.Point(coords[0], coords[1])
    if point.within(circle_buffer) and node in l_genes_dismod_3: 
        l_genes_circlebuffer_3.append(node)
        
print('Number of genes within radius around Center - Disease 3:' ,len(l_genes_circlebuffer_3))

# color edges within circle buffer     
d_edge_col_circlebuffer_3 = color_edges_from_nodelist_specific(G, l_genes_circlebuffer_3, node_col_3)
print('Number of links in circle buffer - Disease 3:', len(d_edge_col_circlebuffer_3))

# color all dismod edges    
d_edge_col_3 = color_edges_from_nodelist_specific(G, l_genes_dismod_3, node_col_3)

Number of genes within radius around Center - Disease 3: 78
Number of links in circle buffer - Disease 3: 71


In [69]:
#
# DISEASE 4 | DARK RED
#

# color nodes
node_col_4 = '#992121' # '#A60C0C' # (darkred)

center_dis_4_sym = 'SLC26A3' #'SCNN1G'
# get nodeID of gene symbol 
for ix, sym in d_gene_sym.items():
    if sym == center_dis_4_sym:
        center_dis_4_id = ix

# get coordinates from nodeID
for node, coords in posG.items():
    if node == center_dis_4_id:
        center_dis_4_coords = coords

# make shapely point from defined center of cluster 
point_4 = geometry.Point(center_dis_4_coords[0], center_dis_4_coords[1])

# create your circle buffer from one of the points
distance = 0.15
circle_buffer = point_4.buffer(distance)

# get list of genes that are within radius around the center 
l_genes_circlebuffer_4 = []
for node,coords in posG.items():
    point = geometry.Point(coords[0], coords[1])
    if point.within(circle_buffer) and node in l_genes_dismod_4: 
        l_genes_circlebuffer_4.append(node)
        
print('Number of genes within radius around Center - Disease 4:' ,len(l_genes_circlebuffer_4))

# color edges within circle buffer     
d_edge_col_circlebuffer_4 = color_edges_from_nodelist_specific(G, l_genes_circlebuffer_4, node_col_4)
print('Number of links in circle buffer - Disease 4:', len(d_edge_col_circlebuffer_4))

# color all dismod edges    
d_edge_col_4 = color_edges_from_nodelist_specific(G, l_genes_dismod_4, node_col_4)

Number of genes within radius around Center - Disease 4: 99
Number of links in circle buffer - Disease 4: 43


### NF SPECIFIC

In [70]:
# -------------------------
#
# Links of NF1 to all its neighbors
#
# -------------------------
node_col_nf = '#0068FF'

d_edge_col_nf1_to_neighbors = color_edges_from_node(G, nf_goi_str, node_col_nf)
#print('Number of links of NF1 neighbors:', len(d_edge_col_nf1_to_neighbors))


# -------------------------
#
# Links of Neurofibromatosis genes to disease module genes 
#
# -------------------------

node_col_nf_diseasemodule_overlap = '#89BBD4'
dismod_all = l_genes_dismod_1 + l_genes_dismod_2 + l_genes_dismod_3 + l_genes_dismod_4

l_genes_nf_diseasemodule_overlap = []
for edge in G.edges():
    if edge[0] in dismod_all and edge[1] in l_genes_nf:
        l_genes_nf_diseasemodule_overlap.append(edge)
    elif edge[1] in dismod_all and edge[0] in l_genes_nf:
        l_genes_nf_diseasemodule_overlap.append(edge)
        
l_selection = []
for edge in l_genes_nf_diseasemodule_overlap:
    for i in edge:
        if i in l_genes_nf or i in dismod_all:
            l_selection.append(i)
        else:
            pass 
        
d_edge_col_nf_diseasemodule_overlap = color_edges_from_nodelist_specific(G, set(l_selection), node_col_nf_diseasemodule_overlap)
print('Number of links of NF1 overlap with disease modules genes (total):', len(d_edge_col_nf_diseasemodule_overlap))

# W I T H  C I R C L E B U F F E R 
center_nf_sym = 'NF1' 

# get nodeID of gene symbol 
for ix, sym in d_gene_sym.items():
    if sym == center_nf_sym:
        center_nf_id = ix
        
# get coordinates from nodeID
for node, coords in posG.items():
    if node == center_nf_id:
        center_nf_coords = coords
        
# make shapely point from defined center of cluster 
point_nf = geometry.Point(center_nf_coords[0], center_nf_coords[1])

# create your circle buffer from one of the points
distance = 0.08
circle_buffer= point_nf.buffer(distance)

# get list of genes that are within radius around the centerpoint
l_genes_circlebuffer_nf_diseasemodule_overlap = []
for node,coords in posG.items():
    point = geometry.Point(coords[0], coords[1])
    if point.within(circle_buffer) and node in l_selection: 
        l_genes_circlebuffer_nf_diseasemodule_overlap.append(node)

d_edge_col_nf_diseasemodule_overlap_buffer = color_edges_from_nodelist_specific(G, l_genes_circlebuffer_nf_diseasemodule_overlap, node_col_nf_diseasemodule_overlap)
print('Number of links of NF1 neighbors to filtered disease module genes:', len(d_edge_col_nf_diseasemodule_overlap_buffer))

Number of links of NF1 overlap with disease modules genes (total): 690
Number of links of NF1 neighbors to filtered disease module genes: 185


---
## PLOT 
---

#### MERGE NODE COLORS > 4 diseases = 4 different colors 

In [71]:
# merge all node colors (and sort like G.nodes)

col_param = list(df_centralities['closeness'].values)
d_node_colors = dict(zip(list(G.nodes()), col_param))

node_col_shared = '#AFC8E8' 

c1='#E6E6E6' 
c2='#9B9B9B' 
n=len(set(col_param))

colors_for_palette = []
for x in range(n+1):
    colors_for_palette.append(colorFader(c1,c2,x/n))    

customPalette = sns.set_palette(sns.color_palette(colors_for_palette))
d_colors = color_nodes_from_dict(G, d_node_colors, palette = customPalette)

d_col_all = {}
for k,v in d_colors.items():
    
    # color for all NF1 associated genes (i.e. neighbors, NF1 disease module) 
    if k in l_genes_nf:# or k in nf_neighbors:
        d_col_all[k] = node_col_nf
   
    #shared genes 1+2
    elif k in l_genes_dismod_1 and k in l_genes_dismod_2:# and k in l_genes_dismod_3 and k in l_genes_dismod_4:
        d_col_all[k] = node_col_shared
        
    #shared genes 1+3
    elif k in l_genes_dismod_1 and k in l_genes_dismod_3:
        d_col_all[k] = node_col_shared
        
    #shared genes 1+4
    elif k in l_genes_dismod_1 and k in l_genes_dismod_4:
        d_col_all[k] = node_col_shared 
        
    #shared genes 2+3
    elif k in l_genes_dismod_2 and k in l_genes_dismod_3:
        d_col_all[k] = node_col_shared
        
    #shared genes 2+4
    elif k in l_genes_dismod_2 and k in l_genes_dismod_4:
        d_col_all[k] = node_col_shared 
        
    #shared genes 3+4
    elif k in l_genes_dismod_3 and k in l_genes_dismod_4:
        d_col_all[k] = node_col_shared 
    
    
    # disease module colors 
    elif k in l_genes_dismod_1:
        d_col_all[k] = node_col_1
        
    elif k in l_genes_dismod_2:
        d_col_all[k] = node_col_2
        
    elif k in l_genes_dismod_3:
        d_col_all[k] = node_col_3
    
    elif k in l_genes_dismod_4:
        d_col_all[k] = node_col_4
    
    else:
        d_col_all[k]=v
        
colors = list(d_col_all.values())

---
## 2D PLOT
---

In [72]:
# --------------------------------------------
# G E N E R A L S E T T I N G S
# --------------------------------------------

opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 0.5
scale_factor = 0.55

# --------------------------------------------
# plot nodes based on focus 
# > separate to background/foreground 
# --------------------------------------------
l_genes_all = l_genes_nf + dismod_all + l_genes_neighbors

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes_all:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree_3D(G, scale_factor/80 ) # 50) #100
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

# --------------------------------------------
# N O D E S 
# --------------------------------------------
umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth, 0.8)
umap_nodes_foreground_ = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, 3, #4, #3,
                                           None, 0.35)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           None,0.5) 
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 2, #2
                                           None,0.5)

# --------------------------------------------
# E D G E S 
# --------------------------------------------
opac_shine = 0.08
lw_shine = 3.0
lw_buffer = 0.25
opac_buffer = 0.5 
umap_edges_circlebuffer_1 = get_trace_edges_specific2D(d_edge_col_circlebuffer_1,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_1_shine = get_trace_edges_specific2D(d_edge_col_circlebuffer_1, posG, linew=lw_shine, opac=opac_shine)

umap_edges_circlebuffer_2 = get_trace_edges_specific2D(d_edge_col_circlebuffer_2,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_2_shine = get_trace_edges_specific2D(d_edge_col_circlebuffer_2,posG, linew=lw_shine, opac=opac_shine)

umap_edges_circlebuffer_3 = get_trace_edges_specific2D(d_edge_col_circlebuffer_3,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_3_shine = get_trace_edges_specific2D(d_edge_col_circlebuffer_3,posG, linew=lw_shine, opac=opac_shine)

umap_edges_circlebuffer_4 = get_trace_edges_specific2D(d_edge_col_circlebuffer_4,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_4_shine = get_trace_edges_specific2D(d_edge_col_circlebuffer_4,posG, linew=lw_shine, opac=opac_shine)

umap_edges_nf_diseasemodule_overlap = get_trace_edges_specific2D(d_edge_col_nf_diseasemodule_overlap_buffer,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_nf_diseasemodule_overlap_shine = get_trace_edges_specific2D(d_edge_col_nf_diseasemodule_overlap_buffer,posG, linew=lw_shine, opac=opac_shine)


opacity_edges = 0.3
width_edges = 0.25
umap_edges_1 = get_trace_edges_specific2D(d_edge_col_1, posG, linew=width_edges, opac=opacity_edges)
umap_edges_2 = get_trace_edges_specific2D(d_edge_col_2, posG, linew=width_edges, opac=opacity_edges)
umap_edges_3 = get_trace_edges_specific2D(d_edge_col_3, posG, linew=width_edges, opac=opacity_edges)
umap_edges_4 = get_trace_edges_specific2D(d_edge_col_4, posG, linew=width_edges, opac=opacity_edges)
                  
    
data = [
    
        ######################
        # nodes background 
        ######################
    
        umap_nodes_background_, 
        umap_nodes_background, 
    
        ######################
        # all edges 
        ######################
        
        #umap_edges_1, 
        #umap_edges_2,
        #umap_edges_3,
        #umap_edges_4,
        
        ######################
        # edges within radius
        ######################
        
        umap_edges_circlebuffer_1_shine,
        umap_edges_circlebuffer_2_shine,
        umap_edges_circlebuffer_3_shine,
        umap_edges_circlebuffer_4_shine,
    
        umap_edges_circlebuffer_1, 
        umap_edges_circlebuffer_2, 
        umap_edges_circlebuffer_3, 
        umap_edges_circlebuffer_4, 
    
        umap_edges_nf_diseasemodule_overlap,
        umap_edges_nf_diseasemodule_overlap_shine,
        
        ######################
        # nodes foreground 
        ######################
        umap_nodes_foreground_,
        umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '2Dportrait_'+'DOIDs'+num_doid_1+'_'+num_doid_2+'_'+num_doid_3+'_'+num_doid_4+'_enh'+str(scalar_val)+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism+'_dismod01linksonly'
#fname = ' test'

plot_2D(data,path,fname)

'output_plots/2Dlayouts/2Dportrait_DOIDs3165_10534_3620_0060089_enh2_nn8_spr1.0_md0.0_cosine_human_dismod01linksonly.html'

---
## 3D PLOT
---

In [60]:
# --------------------------------------------
# G E N E R A L S E T T I N G S
# --------------------------------------------

opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 0.5
scale_factor = 0.55


# --------------------------------------------
# plot nodes based on focus 
# > separate to background/foreground 
# --------------------------------------------
l_genes_all = l_genes_nf + dismod_all + l_genes_neighbors

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes_all:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/50 ) # 50) #100
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

# --------------------------------------------
# N O D E S 
# --------------------------------------------
umap_nodes_foreground = get_trace_nodes_3D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth, 0.8)
umap_nodes_foreground_ = get_trace_nodes_3D(posG_foreground, feat_foreground, colors_foreground, 3, #4, #3,
                                           None, 0.35)
umap_nodes_background = get_trace_nodes_3D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           None,0.5) 
umap_nodes_background_ = get_trace_nodes_3D(posG_background, feat_background, colors_background, 2, #2
                                           None,0.5)

# --------------------------------------------
# E D G E S 
# --------------------------------------------
opac_shine = 0.05
lw_shine = 20.0

lw_buffer = 0.5
opac_buffer = 0.8 

umap_edges_circlebuffer_1 = get_trace_edges_specific3D(d_edge_col_circlebuffer_1,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_1_shine = get_trace_edges_specific3D(d_edge_col_circlebuffer_1, posG, linew=lw_shine, opac=opac_shine)

umap_edges_circlebuffer_2 = get_trace_edges_specific3D(d_edge_col_circlebuffer_2,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_2_shine = get_trace_edges_specific3D(d_edge_col_circlebuffer_2,posG, linew=lw_shine, opac=opac_shine)

umap_edges_circlebuffer_3 = get_trace_edges_specific3D(d_edge_col_circlebuffer_3,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_3_shine = get_trace_edges_specific3D(d_edge_col_circlebuffer_3,posG, linew=lw_shine, opac=opac_shine)

umap_edges_circlebuffer_4 = get_trace_edges_specific3D(d_edge_col_circlebuffer_4,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_circlebuffer_4_shine = get_trace_edges_specific3D(d_edge_col_circlebuffer_4,posG, linew=lw_shine, opac=opac_shine)

umap_edges_nf_diseasemodule_overlap = get_trace_edges_specific3D(d_edge_col_nf_diseasemodule_overlap_buffer,posG, linew=lw_buffer, opac=opac_buffer)
umap_edges_nf_diseasemodule_overlap_shine = get_trace_edges_specific3D(d_edge_col_nf_diseasemodule_overlap_buffer,posG, linew=lw_shine, opac=opac_shine)


opacity_edges = 0.3
width_edges = 0.25
umap_edges_1 = get_trace_edges_specific3D(d_edge_col_1, posG, linew=width_edges, opac=opacity_edges)
umap_edges_2 = get_trace_edges_specific3D(d_edge_col_2, posG, linew=width_edges, opac=opacity_edges)
umap_edges_3 = get_trace_edges_specific3D(d_edge_col_3, posG, linew=width_edges, opac=opacity_edges)
umap_edges_4 = get_trace_edges_specific3D(d_edge_col_4, posG, linew=width_edges, opac=opacity_edges)
                  
    
data = [
    
        ######################
        # nodes background 
        ######################
    
        umap_nodes_background_, 
        umap_nodes_background, 
    
        ######################
        # all edges 
        ######################
        
        #umap_edges_1, 
        #umap_edges_2,
        #umap_edges_3,
        #umap_edges_4,
        
        ######################
        # edges within radius
        ######################
        
        umap_edges_circlebuffer_1_shine,
        umap_edges_circlebuffer_2_shine,
        umap_edges_circlebuffer_3_shine,
        umap_edges_circlebuffer_4_shine,
    
        umap_edges_circlebuffer_1, 
        umap_edges_circlebuffer_2, 
        umap_edges_circlebuffer_3, 
        umap_edges_circlebuffer_4, 
    
        umap_edges_nf_diseasemodule_overlap,
        umap_edges_nf_diseasemodule_overlap_shine,
        
        ######################
        # nodes foreground 
        ######################
        umap_nodes_foreground_,
        umap_nodes_foreground]



path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '3Dportrait_'+'DOIDs'+num_doid_1+'_'+num_doid_2+'_'+num_doid_3+'_'+num_doid_4+'_enh'+str(scalar_val)+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism
#fname = ' test'

plot_3D(data,path,fname)

'output_plots/2Dlayouts/3Dportrait_DOIDs3165_10534_3620_0060089_enh2_nn8_spr0.9_md0.0_cosine_human.html'

### EXPORT POSITIONS >  VRNETZER Format 

In [73]:
path = 'output_plots/VRlayouts/' 
layout_nmspce = '2Dportrait_NF1'
cols = colors
export_to_csv2D(path, layout_nmspce, posG, cols)
#export_to_csv3D(path, layout_nmspce, posG, cols)

---
# ADDITIONAL | NF1 + 8712 + NEIGHBORS OF NF1
---
highlighting Neurofibromatosis + neighbors of gene NF1

In [ ]:
# merge all node colors (and sort like G.nodes)

col_param = list(df_centralities['closeness'].values)
d_node_colors = dict(zip(list(G.nodes()), col_param))

c1='#E6E6E6' 
c2='#9B9B9B' 
n=len(set(col_param))

colors_for_palette = []
for x in range(n+1):
    colors_for_palette.append(colorFader(c1,c2,x/n))    

customPalette = sns.set_palette(sns.color_palette(colors_for_palette))
d_colors = color_nodes_from_dict(G, d_node_colors, palette = customPalette)

d_col_all = {}
for k,v in d_colors.items():
    
    if k in l_genes_dismod:
        d_col_all[k] = node_col_dismod

    elif k in nf_neighbors: 
        d_col_all[k] = node_col_dismod
    
    else:
        d_col_all[k]=v
        
colors = list(d_col_all.values())

In [ ]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes_dismod:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())


umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth*0.25, 0.8)
umap_nodes_foreground_ = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, 5,
                                           None, 0.2)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           None,0.5) #node_linewidth*0.05, 0.5)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 2, 
                                            None,0.5)

umap_edges_dismod = get_trace_edges_specific2D(d_edge_col_dismod, posG, linew=width_edges, 
                                               opac=0.5)# opacity_edges)
umap_edges_dismod_neigh = get_trace_edges_specific2D(d_edge_col_neigh, posG, linew=width_edges, 
                                                     opac=0.5) #opacity_edges)

data = [
        umap_edges_dismod,
        umap_edges_dismod_neigh,
        umap_nodes_background_, 
        umap_nodes_background, 
        umap_nodes_foreground_,
        umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname_NF1links = '2Dportrait_NF8712_'+'DOIDs'+num_doid_1+'_'+num_doid_2+'_'+num_doid_3+'_'+num_doid_4+'_enhDismod'+str(scalar_val_dismod)+'_enhContext'+str(scalar_val_discontext)+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism+'_NF1links'

plot_2D(data,path,fname_NF1links)

---
# ADDITIONAL | ONLY NF1 + 8712
---
highlighting Neurofibromatosis

In [ ]:
col_param = list(df_centralities['closeness'].values)
d_node_colors = dict(zip(list(G.nodes()), col_param))

c1='#E6E6E6' 
c2='#9B9B9B' 
n=len(set(col_param))

colors_for_palette = []
for x in range(n+1):
    colors_for_palette.append(colorFader(c1,c2,x/n))    

customPalette = sns.set_palette(sns.color_palette(colors_for_palette))
d_colors = color_nodes_from_dict(G, d_node_colors, palette = customPalette)

d_col_all = {}
for k,v in d_colors.items():
    if k in l_genes_dismod:
        d_col_all[k] = node_col_dismod
    
    else:
        d_col_all[k]=v
        
colors = list(d_col_all.values())

In [ ]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes_dismod:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())


umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth*0.25, 0.8)
umap_nodes_foreground_ = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, 5,
                                           None, 0.2)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           None,0.5) #node_linewidth*0.05, 0.5)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 2, 
                                            None,0.5)

umap_edges_dismod = get_trace_edges_specific2D(d_edge_col_dismod, posG, linew=0.5,#width_edges, 
                                               opac=0.9) #opacity_edges)

data = [
        #umap_edges_dismod,
        umap_nodes_background_, 
        umap_nodes_background, 
        umap_nodes_foreground_,
        umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname_NF1module = '2Dportrait_NF8712_'+'DOIDs'+num_doid_1+'_'+num_doid_2+'_'+num_doid_3+'_'+num_doid_4+'_enhDismod'+str(scalar_val_dismod)+'_enhContext'+str(scalar_val_discontext)+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism+'_NF1module_nolinks'

plot_2D(data,path,fname_NF1module)